# Assignment 1 – Backprop

In [ ]:
#@title Library Imports

import importlib 
!git clone https://www.github.com/rycolab/intro-nlp-f22.git
utils = importlib.import_module("intro-nlp-f22.assignment_1.utils")

import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

Cloning into 'intro-nlp-f22'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 6), reused 30 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
#@title Select and Parse Math Problems

#@markdown select math problem

math_problem_i = "3" #@param [0,1,2,3]
math_problem = utils.MATH_PROBLEMS[int(math_problem_i)]
print(math_problem)

parser = utils.Parser()
infix, in_vars = parser.parse(math_problem["problem"], in_vars = math_problem["in_vars"])
print(infix, in_vars)

{'problem': 'z + sin(x^(2) + (y * exp(z)))', 'in_vars': {'x': 2.0, 'y': -1.0, 'z': 0.0}, 'output': 0.14, 'derivative': {'x': -3.96, 'y': -0.99, 'z': 1.99}}
['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]] {'x': 2.0, 'y': -1.0, 'z': 0.0}


In [ ]:
#@title 1: Building

class Builder():

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)

        self.infix = infix

        operations = ["log", "exp", "+", "-", "^", "sin", "cos", "*", "/"]
        graph = {}
        graph2 = {}
        infix2 = [infix]
        k = 0
        i = 0
        while i<len(infix2):
          temp = infix2[i]
          for el in temp:
            if isinstance(el, list):
              infix2 = infix2 + [el]
            if el not in operations:
              for el2 in temp:
                if el2 in operations:
                  graph[self.avail_vars[k]] = [str(el),temp, el2]
                  k = k + 1
          i = i + 1

        self.graph = graph

if __name__ == '__main__':

    g = Builder(infix, in_vars)
    print(g.graph)

{'ψ': ['z', ['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]], '+'], 'χ': ["['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]", ['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]], '+'], 'φ': ["[['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]", ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]], 'sin'], 'υ': ["['x', '^', 2]", [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]], '+'], 'τ': ["['y', '*', ['exp', 'z']]", [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]], '+'], 'σ': ['x', ['x', '^', 2], '^'], 'ς': ['2', ['x', '^', 2], '^'], 'ρ': ['y', ['y', '*', ['exp', 'z']], '*'], 'π': ["['exp', 'z']", ['y', '*', ['exp', 'z']], '*'], 'ο': ['z', ['exp', 'z'], 'exp']}


In [ ]:
#@title 2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        return math.log(a)

    def df(self, a, b = None):
        return [1 / a]

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b):
        return [b, a]

class Div(Operator):

    def f(self, a, b):
        return a / b

    def df(self, a, b):
        return [1 / b, -a / (b**2)]

class Add(Operator):

    def f(self, a, b):
        return a + b

    def df(self, a, b):
        return [1, 1]

class Sub(Operator):

    def f(self, a, b):
        return a - b

    def df(self, a, b):
        return [1, -1]

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None):
        return math.sin(a)

    def df(self, a, b=None):
        return [math.cos(a)]

class Cos(Operator):

    def f(self, a, b=None):
        return math.cos(a)

    def df(self, a, b=None):
        return [-math.sin(a)]

In [ ]:
#@title 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________
    def forward(self, ):
        in_vars = self.in_vars
        operations = self.fn_map
        graph = self.graph
        results = {}
        results.update(in_vars)
        chiavi = sorted(graph.keys())
        flag = 0
        for el in chiavi:
          if len(graph[el][1])>2: # if the operation has two inputs
            for el2 in graph[el][1]:
              if el2 != graph[el][0] and str(el2) not in operations: # look for second input
                if isinstance(graph[el][0],int) and isinstance(el2,int):
                  results[str(graph[el][1])] = operations[graph[el][2]].f(int(graph[el][0]),int(el2))
                elif str(el2) in results.keys() or isinstance(el2, list): # if second input is variable/list
                  if str(graph[el][0]) in results.keys():
                    results[str(graph[el][1])] = operations[graph[el][2]].f(results[str(graph[el][0])],results[str(el2)])
                  else:
                    results[str(graph[el][1])] = operations[graph[el][2]].f(eval(graph[el][0]),results[str(el2)]) #float
                elif isinstance(el2, float) or isinstance(el2, int): # if second input is a float
                  if str(graph[el][0]) in results.keys():
                    results[str(graph[el][1])] = operations[graph[el][2]].f(results[str(graph[el][0])],el2)
                  else:
                    results[str(graph[el][1])] = operations[graph[el][2]].f(eval(graph[el][0]),el2) #float
          else: # if the operation has one input
            if str(graph[el][0]) in results.keys():
              results[str(graph[el][1])] = self.fn_map[graph[el][2]].f(results[graph[el][0]])
            else:
              results[str(graph[el][1])] = self.fn_map[graph[el][2]].f(eval(graph[el][0])) #float
        self.results = results
        self.output = results[str(self.graph[el][1])]
   
    ## backward execution____________________________
    def backward(self, ):
        in_vars = self.in_vars
        results = self.results
        deriv = {}
        lst = []
        for el in results.keys():
          lst.insert(0,el)
          deriv[el] = 0
        deriv[lst[0]] = 1
        for el in lst: 
          if el not in in_vars.keys():
            el = eval(el)
          if len(el) == 3:
            el0 = str(el[0])
            el2 = str(el[2])
            if isinstance(el[0],list)==False and el[0] not in results:
              results[el0] = eval(el0)
              deriv[el0] = eval(el0)
            if isinstance(el[2],list)==False and el[2] not in results:
              results[el2] = eval(el2)
              deriv[el2] = eval(el2)
            deriv[str(el[0])] += deriv[str(el)] * self.fn_map[el[1]].df(results[str(el[0])],results[str(el[2])])[0]
            deriv[str(el[2])] += deriv[str(el)] * self.fn_map[el[1]].df(results[str(el[0])],results[str(el[2])])[1]
          elif len(el) == 2:
            if el[0] in results.keys():
              deriv[str(el[0])] += deriv[str(el)] * self.fn_map[str(el[1])].df(results[str(el[0])])[0]
            else:
              deriv[str(el[1])] += deriv[str(el)] * self.fn_map[str(el[0])].df(results[str(el[1])])[0]
          else:
            True 

        gradient = {}
        for el in in_vars:
          gradient[el] = deriv[el]
        self.derivative = gradient

if __name__ == '__main__':
  e = Executor(g.graph, in_vars=in_vars)
  e.forward()
  e.backward()
  print(e.output)
  print(e.derivative)

0.1411200080598672
{'x': -3.9599699864017817, 'y': -0.9899924966004454, 'z': 1.9899924966004454}


In [ ]:
#@title Test Function for Debugging

utils.test_backprop(Builder, Executor, math_problem)


0: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
SUCCESS output: 0.14
SUCCESS derivative: {'x': -3.96, 'y': -0.99, 'z': 1.99}


In [ ]:
#@title Test Function for Grading

utils.test_backprop(Builder, Executor)


0: problem: x/y, in_vars: {'x': 1.0, 'y': 1.0}
SUCCESS output: 1.0
SUCCESS derivative: {'x': 1.0, 'y': -1.0}

1: problem: exp(x) - (y * 2), in_vars: {'x': 2.0, 'y': -2.0}
SUCCESS output: 11.39
SUCCESS derivative: {'x': 7.39, 'y': -2}

2: problem: (x^2 - 1) * (y+2), in_vars: {'x': 3.0, 'y': 2.0}
SUCCESS output: 32.0
SUCCESS derivative: {'x': 24.0, 'y': 8.0}

3: problem: z + sin(x^(2) + (y * exp(z))), in_vars: {'x': 2.0, 'y': -1.0, 'z': 0.0}
SUCCESS output: 0.14
SUCCESS derivative: {'x': -3.96, 'y': -0.99, 'z': 1.99}
